<a href="https://colab.research.google.com/github/Fernando-Hillesheim/Learning-ML/blob/main/Technics_to_improve_training_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Normalizing Data

Manualy normalizing

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

In [ ]:
class_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

Using batch normalization layer

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
  ])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Changing Optimizers

There are three ways to change optimizers, let's take a look!

In [ ]:
#1º
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

#2º
opt = keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

#3º
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=0.001), metrics=["accuracy"])

# Learning Rate Scheduling

In [ ]:
#Power scheduling
opt = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

#Exponencial scheduling
def exponencial_decay(epoch):
    return 0.01 * 0.1**(epoch/20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponencial_decay)

  #or

initial_learning_rate = 0.1
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96)

opt = keras.optimizers.SGD(learning_rate=lr_scheduler)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

#Piecewise constant scheduling
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

  #or

step = tf.Variable(0, trainable=False)
boundaries = [100000, 150000]
values = [1.0, 0.1, 0.01]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)
learning_rate = learning_rate_fn(step)

lr_scheduler = keras.callbacks.LearningRateScheduler(learning_rate)

#Perfornance scheduler
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.001)


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 200)            │        60,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 319,946 (1.22 MB)

 Trainable params: 318,378 (1.21 MB)

 Non-trainable params: 1,568 (6.12 KB)

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[lr_scheduler])

Epoch 1/30
1131/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8597 - loss: 0.4467

KeyboardInterrupt: 

# Pruning the Network

In [ ]:
import tempfile
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10),
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_images, train_labels, epochs=4, validation_split=0.1)

Epoch 1/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.8489 - loss: 0.5337 - val_accuracy: 0.9610 - val_loss: 0.1443
Epoch 2/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9546 - loss: 0.1576 - val_accuracy: 0.9733 - val_loss: 0.0980
Epoch 3/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.9705 - loss: 0.1038 - val_accuracy: 0.9798 - val_loss: 0.0794
Epoch 4/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.9779 - loss: 0.0764 - val_accuracy: 0.9813 - val_loss: 0.0704


In [ ]:
_, baseline_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.9768999814987183


In [ ]:
import tensorflow_model_optimization as tfmot
#not working - material link: https://www.youtube.com/watch?v=NBhas7xb3h4&list=PLM8lYG2MzHmQn55ii0duXdO9QSoDF5myF&index=31
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model)

# 'prune_low_magnitude' requires recompile.
model_for_pruning.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model_for_pruning.summary()

ValueError: `prune_low_magnitude` can only prune an object of the following types: keras.models.Sequential, keras functional model, keras.layers.Layer, list of keras.layers.Layer. You passed an object of type: Sequential.